In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split
import ipynb.fs.full.features as features
import ipynb.fs.full.utils as utils
from hyperopt import fmin, tpe, hp
from hyperopt.pyll.base import scope
from hyperopt import STATUS_OK
from sklearn.metrics import mean_absolute_error
import ipynb.fs.full.features_distancias as f_distancias

df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')

df_train_idf = pd.read_csv('./data/train_idf.csv')
df_test_idf = pd.read_csv('./data/test_idf.csv')

df_train = pd.merge(df_train, df_train_idf, on= 'id', how= 'left')
df_test = pd.merge(df_test, df_test_idf, on= 'id', how= 'left')

df_train_idf = pd.read_csv('./data/train_idf.csv')
df_test_idf = pd.read_csv('./data/test_idf.csv')

df_train = pd.merge(df_train, df_train_idf, on= 'id', how= 'left')
df_test = pd.merge(df_test, df_test_idf, on= 'id', how= 'left')


df_train = features.llenar_nulls(df_train)
df_test = features.llenar_nulls(df_test)

### Agrego Features

In [ ]:
df_test_f = features.features_independientes_precio(df_test)
df_test_f = features.features_dependientes_precio(df_test_f, df_train)

df_train_f = features.features_independientes_precio(df_train)
df_train_f = features.features_dependientes_precio(df_train_f, df_train)

df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', df_aux=df_train_f, devolver_cols=True)

df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'zona', df_aux=df_test_f)

df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)


df_train_f = f_distancias.feature_distancias(df_train_f)
df_test_f = f_distancias.feature_distancias(df_test_f, df_train_f)

df_train_f = features.rankings(df_train_f, df_train_f)
df_test_f = features.rankings(df_test_f, df_train_f)

### Filtro columnas

In [4]:
features = ['antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 
            'lat', 'lng',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['prop_frecuente', 'top_provincia', 'porcentaje_metros', 'diferencia_metros', 'promedio_precio_ciudad', 
                 'promedio_por_mes', 'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 
                 'promedio_precio_hbg_tipo_propiedad', 'count_id_zona', 'count_ciudad', 'puntaje', 
                 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 
                 'promedio_precio_tipo_propiedad_ciudad_gen', 'promedio_precio_hbg_tipo_propiedad_provincia',
                 'varianza_id_zona', 'tam_ambientes', 'metros_cubiertos_normalizados', 'dias_desde_datos',
                 'meses_desde_datos', 'promedio_id_zona_log']

features += features_test

features += cols_tipodepropiedad_ohe + cols_provincia_ohe + cols_zona_ohe

df_XGBoost, df_eval = utils.dividir_df_testeo(df_train_f, test_size=0.15)

df_XGBoost = utils.filtrar_features(df_XGBoost, features, 'precio')
df_eval = utils.filtrar_features(df_eval, features, 'precio')

## Busco los mejores hiperparametros

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import xgboost as xgb

class XGBoostWrapper(xgb.XGBRegressor):
    
    def fit(self, x, y):
        return super(xgb.XGBRegressor, self).fit(x, y)
    
    def predict(self, X):
        return super(xgb.XGBRegressor, self).predict(X)


gs = joblib.load('xgboost_gscv3.pkl')

In [15]:
results = pd.DataFrame(gs.cv_results_).sort_values('rank_test_score')
results.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_colsample_bytree', 'param_eval_metric', 'param_gamma',
       'param_learning_rate', 'param_max_depth', 'param_n_estimators',
       'param_n_jobs', 'param_objective', 'param_scale_pos_weight',
       'param_subsample', 'param_verbosity', 'params', 'split0_test_score',
       'split1_test_score', 'split2_test_score', 'split3_test_score',
       'mean_test_score', 'std_test_score', 'rank_test_score'],
      dtype='object')

In [17]:
results[['mean_fit_time', 'param_colsample_bytree', 'param_eval_metric', 'param_gamma',
       'param_learning_rate', 'param_max_depth', 'param_n_estimators',
       'param_n_jobs', 'param_objective', 'param_scale_pos_weight',
       'param_subsample', 'rank_test_score']].head(30)

,mean_fit_time,param_colsample_bytree,param_eval_metric,param_gamma,param_learning_rate,param_max_depth,param_n_estimators,param_n_jobs,param_objective,param_scale_pos_weight,param_subsample,rank_test_score
701,37.456315,0.9,mae,0,0.1,10,70,4,reg:squarederror,4,0.9,1
917,37.836063,0.9,mae,0.3,0.1,10,70,4,reg:squarederror,4,0.9,1
914,37.720415,0.9,mae,0.3,0.1,10,70,4,reg:squarederror,2,0.9,1
803,37.437682,0.9,mae,0.1,0.1,10,70,4,reg:squarederror,1,0.9,1
806,37.277326,0.9,mae,0.1,0.1,10,70,4,reg:squarederror,2,0.9,1
911,37.435519,0.9,mae,0.3,0.1,10,70,4,reg:squarederror,1,0.9,1
698,37.448860,0.9,mae,0,0.1,10,70,4,reg:squarederror,2,0.9,1
809,38.375836,0.9,mae,0.1,0.1,10,70,4,reg:squarederror,4,0.9,1
695,37.220169,0.9,mae,0,0.1,10,70,4,reg:squarederror,1,0.9,1
374,31.740836,0.7,mae,0,0.1,10,70,4,reg:squarederror,2,0.9,10


In [4]:
from sklearn.externals import joblib

def full_display(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
        display(df)
        
def mostrar_top_n(file, n):
    full_display(pd.DataFrame(joblib.load(file).cv_results_).sort_values('rank_test_score').head(n))
    
mostrar_top_n('xgboost_rscv_1.pkl', 3)
mostrar_top_n('xgboost_rscv_2.pkl', 3)
mostrar_top_n('xgboost_rscv_3.pkl', 3)
mostrar_top_n('xgboost_rscv_4.pkl', 3)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_verbosity,param_subsample,param_scale_pos_weight,param_reg_lambda,param_reg_alpha,param_objective,param_n_jobs,param_n_estimators,param_max_depth,param_learning_rate,param_eval_metric,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
8,21.572635,1.172920,0.379922,0.002603,0,0.9,2,1,15,reg:squarederror,4,30,15,0.1,mae,0.5,"{'verbosity': 0, 'subsample': 0.9, 'scale_pos_...",-494020.155144,-490647.328515,-488784.157059,-493581.164986,-491758.206297,2151.659827,1
45,36.550329,2.334331,0.377513,0.022408,0,0.7,4,1,1,reg:squarederror,4,30,15,0.1,mae,0.9,"{'verbosity': 0, 'subsample': 0.7, 'scale_pos_...",-498969.376332,-493435.967544,-488194.571405,-494670.754896,-493817.687731,3841.637627,2
34,21.963166,0.990556,0.338900,0.003660,0,0.9,1,4,30,reg:squarederror,4,30,15,0.1,mae,0.5,"{'verbosity': 0, 'subsample': 0.9, 'scale_pos_...",-499064.425287,-495483.887533,-491708.355276,-497733.363091,-495997.518602,2787.459069,3


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_verbosity,param_subsample,param_scale_pos_weight,param_reg_alpha,param_objective,param_n_jobs,param_n_estimators,param_max_depth,param_learning_rate,param_eval_metric,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
17,28.076588,1.290356,0.408893,0.025659,0,0.9,1,0,reg:squarederror,4,30,15,0.1,mae,0.7,"{'verbosity': 0, 'subsample': 0.9, 'scale_pos_...",-493530.449709,-490071.925847,-487218.297118,-491538.323837,-490589.759381,2301.212459,1
25,34.067499,1.368834,0.379427,0.016048,0,0.9,4,4,reg:squarederror,4,30,15,0.1,mae,0.9,"{'verbosity': 0, 'subsample': 0.9, 'scale_pos_...",-494035.905043,-490052.292257,-487331.509508,-492275.687460,-490923.858049,2508.826315,2
13,27.331998,0.205388,0.406695,0.036679,0,0.9,4,15,reg:squarederror,4,30,15,0.1,mae,0.7,"{'verbosity': 0, 'subsample': 0.9, 'scale_pos_...",-494122.937223,-489800.172553,-487130.545708,-492724.415411,-490944.526332,2698.436647,3


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_verbosity,param_subsample,param_scale_pos_weight,param_reg_alpha,param_objective,param_n_jobs,param_n_estimators,param_max_depth,param_learning_rate,param_eval_metric,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
143,28.279560,1.121788,0.376085,0.006590,0,0.9,2,4,reg:squarederror,4,30,15,0.1,mae,0.7,"{'verbosity': 0, 'subsample': 0.9, 'scale_pos_...",-493933.509453,-489965.416247,-486540.481780,-491802.681901,-490560.534101,2712.693737,1
51,28.112561,1.346262,0.369538,0.010110,0,0.9,2,0,reg:squarederror,4,30,15,0.1,mae,0.7,"{'verbosity': 0, 'subsample': 0.9, 'scale_pos_...",-493530.449709,-490071.925847,-487218.297118,-491538.323837,-490589.759381,2301.212459,2
191,28.095123,1.409538,0.377965,0.011265,0,0.9,1,0,reg:squarederror,4,30,15,0.1,mae,0.7,"{'verbosity': 0, 'subsample': 0.9, 'scale_pos_...",-493530.449709,-490071.925847,-487218.297118,-491538.323837,-490589.759381,2301.212459,2


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_verbosity,param_subsample,param_scale_pos_weight,param_reg_lambda,param_reg_alpha,param_objective,param_n_jobs,param_n_estimators,param_max_depth,param_learning_rate,param_eval_metric,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
14,29.777502,0.283654,0.458209,0.002741,0,0.9,4,1,15,reg:squarederror,4,30,15,0.1,mae,0.7,"{'verbosity': 0, 'subsample': 0.9, 'scale_pos_...",-491856.050710,-499798.953211,-497082.894169,-498514.774264,-496813.160999,3018.950351,1
112,29.925094,0.377395,0.454253,0.002623,0,0.9,2,1,10,reg:squarederror,4,30,15,0.1,mae,0.7,"{'verbosity': 0, 'subsample': 0.9, 'scale_pos_...",-491952.919066,-500663.575709,-496882.314728,-498281.757936,-496945.136290,3183.538426,2
107,29.991858,0.281536,0.452808,0.003048,0,0.9,1,1,0,reg:squarederror,4,30,15,0.1,mae,0.7,"{'verbosity': 0, 'subsample': 0.9, 'scale_pos_...",-491436.954937,-500541.094368,-497495.873078,-498321.678670,-496948.894543,3371.544810,3


In [6]:
pd.DataFrame(joblib.load('xgboost_rscv_3.pkl').cv_results_).sort_values('rank_test_score').iloc[0]

mean_fit_time                                                       28.2796
std_fit_time                                                        1.12179
mean_score_time                                                    0.376085
std_score_time                                                    0.0065898
param_verbosity                                                           0
param_subsample                                                         0.9
param_scale_pos_weight                                                    2
param_reg_alpha                                                           4
param_objective                                            reg:squarederror
param_n_jobs                                                              4
param_n_estimators                                                       30
param_max_depth                                                          15
param_learning_rate                                                     0.1
param_eval_m

### hyperopt

In [ ]:
from sklearn.model_selection import cross_val_score

features = ['habitaciones', 
            'garages', 
            'banos',
            'antiguedad',
           'metroscubiertos', 
            'metrostotales',
            'lat_norm', 'lng_norm'
            'gimnasio', 'usosmultiples', 'piscina','prop_frecuente', 'top_provincia', 'promedio_precio_ciudad', 
                 'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 
                 'count_id_zona', 'count_ciudad', 'puntaje', 
                     'count_tipo_propiedad_ciudad', 
                 'promedio_precio_tipo_propiedad_ciudad_gen',
                 'count_id_zona'
                 'dias_desde_datos',
                 'meses_desde_datos',
                 'porcentaje_metros',
                 'distancia_ciudad_centrica', 'distancia_centro_mexico']


x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=2)


def eval_xgboost(params):
    return {'loss': -np.average(cross_val_score(xgb.XGBRegressor(**params), x_train, y_train, cv=3, scoring='neg_mean_absolute_error')),
            'status': STATUS_OK}

space = {
    'colsample_bytree': hp.uniform('colsample_bytree', 0.20, 1), 
    'max_depth': scope.int(hp.quniform('max_depth', 5, 15, 1)),
    'n_estimators': 25,
    'learning_rate': 0.15,
    'scale_pos_weight': hp.quniform('scale_pos_weight', 5, 15, 1),
    'n_jobs': 4,
    'reg_alpha': hp.uniform('reg_alpha', 0, 50),
    'subsample': hp.uniform('subsample', 0.3, 1),
    'objective': 'reg:squarederror',
    'eval_metric': 'mae',
}

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

hps = fmin(eval_xgboost, space=space, algo=tpe.suggest, max_evals=100, verbose=1)

display(hps)

### Busco los mejores features

In [ ]:
features = ['antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 
            # 'lat', 'lng',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['tipo_propiedad_compartida',
                 'prop_frecuente', 'top_provincia', 'es_ciudad_centrica',
                 'promedio_metros_totales_provincia',
                 'promedio_metros_cubiertos_provincia', 'porcentaje_metros',
                 'diferencia_metros', 'delincuencia', 'turismo',
                 'promedio_precio_ciudad', 'promedio_id_zona',
                 'promedio_precio_tipo_propiedad','es_antigua',
                 'promedio_por_mes', 'promedio_precio_habitaciones',
                 'promedio_precio_habitaciones_banos_garages','cantidad_inquilinos',
                 'promedio_precio_banos_garages', 'promedio_precio_booleanos',
                 'metros_totales_normalizados', 'metros_cubiertos_normalizados','escomercial',
                 'promedio_metros_cub_tipo_propiedad', 'anio', 'mes', 'dia','trimestre']


In [ ]:
hps = {'alpha': 9.499855511978337,
 'colsample_bytree': 0.8500000000000001,
 'learning_rate': 0.2226513740962932,
 'max_depth': 20.0,
 'n_estimators': 240.0,
 'test_size': 0.103950169925154826
}

alpha = hps['alpha']
colsample_bytree = hps['colsample_bytree']
max_depth = hps['max_depth']
learning_rate = hps['learning_rate']
n_estimators = hps['n_estimators']

n_estimators = int(hps['n_estimators'])
max_depth = int(hps['max_depth'])

params = {
        'objective':'reg:squarederror', 
        'colsample_bytree' : colsample_bytree , 
        'learning_rate' : learning_rate,
        'max_depth' : max_depth, 
        'alpha' : alpha,
        'n_estimators' : n_estimators,
        'eval_metric': 'mae'
}


base_train = 0
base_test = 0
base_eval = 0
for i in ['None'] + features_test:
    x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features + [i])

    xg_train = xgb.DMatrix(x_train, label=y_train)
    xg_test = xgb.DMatrix(x_test, label=y_test)
    xg_train = xgb.DMatrix(x_train, label=y_train)
    xg_test = xgb.DMatrix(x_test, label=y_test)

    watchlist = [(xg_train, 'train'), (xg_test, 'test')]
    num_round = 10
    
    xg_reg = xgb.train(params, 
                    xg_train, 
                    num_round, 
                    watchlist
                   )
    
    x_train_predict = xgb.DMatrix(x_train, label=y_train)
    x_test_predict = xgb.DMatrix(x_test, label=y_test)
    x_eval_predict = xgb.DMatrix(utils.filtrar_features(df_eval, features + [i]))
    
    y_pred_test = xg_reg.predict(x_test_predict)
    y_pred_train = xg_reg.predict(x_train_predict)
    y_pred_eval = xg_reg.predict(x_eval_predict)

    xgb_mae_train = utils.MAE(y_train, y_pred_train)
    xgb_mae = utils.MAE(y_test, y_pred_test)
    xgb_mae_eval = utils.MAE(df_eval['precio'].values, y_pred_eval)

    print(f"MAE LightGBM (train): {xgb_mae_train:.5f}")
    print(f"MAE LightGBM (test): {xgb_mae:.5f}")
    print(f"MAE LightGBM (eval): {xgb_mae_eval:.5f}")
    if i is not 'None':
        print(f"Overfitting (base_eval - base_test) - (eval - test) - {i}: {(base_eval - base_test) - (xgb_mae_eval - xgb_mae)}")
        print(f"Diff evaluation (base_eval - eval)                  - {i}: {base_eval - xgb_mae_eval}")
        print(f"Diff train (base_train - train)                     - {i}: {base_train - xgb_mae_train}")
    else:
        base_train = xgb_mae_train
        base_test = xgb_mae
        base_eval = xgb_mae_eval


## Predicción

In [8]:
features = ['antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 
            'lat', 'lng',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['prop_frecuente', 'top_provincia', 'porcentaje_metros', 'diferencia_metros', 'promedio_precio_ciudad', 
                 'promedio_por_mes', 'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 
                 'promedio_precio_hbg_tipo_propiedad', 'count_id_zona', 'count_ciudad', 'puntaje', 
                 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 
                 'promedio_precio_tipo_propiedad_ciudad_gen', 'promedio_precio_hbg_tipo_propiedad_provincia',
                 'varianza_id_zona', 'tam_ambientes', 'metros_cubiertos_normalizados', 'dias_desde_datos',
                 'meses_desde_datos', 'promedio_id_zona_log']

features += features_test

features += cols_tipodepropiedad_ohe + cols_provincia_ohe + cols_zona_ohe

# Preparación hiperparámetros

# hps = {'alpha': 12.338515720263425,
#  'colsample_bytree': 0.8500000000000001,
#  'learning_rate': 0.15,
#  'max_depth': 17.0,
#  'n_estimators': 300.0,
#  'test_size': 0.15000000000000002}

hps = {'alpha': 19.99916025229205,
 'colsample_bytree': 0.65,
 'learning_rate': 0.15,
 'max_depth': int(20.0),
 'n_estimators': int(90.0),
 'test_size': 0.1}

alpha = hps['alpha']
colsample_bytree = hps['colsample_bytree']
max_depth = hps['max_depth']
learning_rate = hps['learning_rate']
n_estimators = hps['n_estimators']
test_size = hps['test_size']

n_estimators = int(hps['n_estimators'])
max_depth = int(hps['max_depth'])


# Preparacion de los datos
x_train, x_test, y_train, y_test = utils.dividir_dataset(df_XGBoost, 'precio', features, test_size=test_size)

x_eval = df_eval.drop('precio', axis=1)
y_eval = df_eval['precio']

xg_train = xgb.DMatrix(x_train, label=y_train)
xg_test = xgb.DMatrix(x_test, label=y_test)
dfg_test = xgb.DMatrix(x_eval, label = y_eval)


params = {
    'objective':'reg:squarederror', 
    'colsample_bytree' : colsample_bytree , 
    'learning_rate' : learning_rate,
    'max_depth' : max_depth, 
    'alpha' : alpha,
    'n_estimators' : n_estimators,
    'eval_metric': 'mae'
}

watchlist = [(xg_train, 'train'), (xg_test, 'test')]

# Entrenamiento
xg_reg = xgb.train(params, 
                   xg_train, 
                   num_boost_round=50,
                   evals=watchlist)

# Prediccion
y_pred_train = xg_reg.predict(xg_train)
y_pred_test = xg_reg.predict(xg_test)


linear_mae_train = utils.MAE(y_train, y_pred_train)
linear_mae = utils.MAE(y_test, y_pred_test)

y_pred_eval = xg_reg.predict(dfg_test)
linear_mae_eval = utils.MAE(y_eval, y_pred_eval)


print(f"MAE (train): {linear_mae_train:.5f}")
print(f"MAE: {linear_mae:.5f}")
print(f"MAE (eval): {linear_mae_eval:.5f}")


/home/javier/Documents/FIUBA/Datos/.venv/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/javier/Documents/FIUBA/Datos/.venv/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[0]	train-mae:2.16263e+06	test-mae:2.18108e+06
[1]	train-mae:1.84763e+06	test-mae:1.86739e+06
[2]	train-mae:1.58015e+06	test-mae:1.60335e+06
[3]	train-mae:1.35271e+06	test-mae:1.38282e+06
[4]	train-mae:1.1596e+06	test-mae:1.19971e+06
[5]	train-mae:996237	test-mae:1.05124e+06
[6]	train-mae:857518	test-mae:930787
[7]	train-mae:738939	test-mae:833015
[8]	train-mae:638519	test-mae:755706
[9]	train-mae:552710	test-mae:694137
[10]	train-mae:479926	test-mae:647447
[11]	train-mae:417837	test-mae:610690
[12]	train-mae:364434	test-mae:581549
[13]	train-mae:318952	test-mae:559763
[14]	train-mae:279914	test-mae:542763
[15]	train-mae:246921	test-mae:530034
[16]	train-mae:218572	test-mae:519891
[17]	train-mae:194438	test-mae:512738
[18]	train-mae:173315	test-mae:506899
[19]	train-mae:155467	test-mae:502224
[20]	train-mae:140111	test-mae:498859
[21]	train-mae:126857	test-mae:496404
[22]	train-mae:115677	test-mae:494486
[23]	train-mae:105637	test-mae:492864
[24]	train-mae:96929.6	test-mae:491596
[25]	

## Predicción final

In [4]:
df_train_final = pd.read_csv('./data/train.csv')
df_test_final = pd.read_csv('./data/test.csv')

df_train_final = features.llenar_nulls(df_train_final)
df_test_final = features.llenar_nulls(df_test_final)

df_train_final_f = features.features_independientes_precio(df_train_final)
df_train_final_f = features.features_dependientes_precio(df_train_final_f, df_train_final)
df_test_final_f = features.features_independientes_precio(df_test_final)
df_test_final_f = features.features_dependientes_precio(df_test_final_f, df_train_final)

df_test_final_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_final_f, 'tipodepropiedad', N=100, df_aux=df_train_final, devolver_cols=True)
df_test_final_f, cols_provincia_ohe = features.columna_a_ohe(df_test_final_f, 'provincia', N=100, df_aux=df_train_final, devolver_cols=True)
df_test_final_f, cols_zona_ohe = features.columna_a_ohe(df_test_final_f, 'zona', df_aux=df_train_final_f, devolver_cols=True)

df_train_final_f = features.columna_a_ohe(df_train_final_f, 'tipodepropiedad', N=100, df_aux=df_test_final)
df_train_final_f = features.columna_a_ohe(df_train_final_f, 'provincia', N=100, df_aux=df_test_final)
df_train_final_f = features.columna_a_ohe(df_train_final_f, 'zona', df_aux=df_test_final_f)

df_train_final_f['fecha'] = pd.to_datetime(df_train_final_f['fecha']).astype(int)
df_test_final_f['fecha'] = pd.to_datetime(df_test_final_f['fecha']).astype(int)

In [8]:
features = ['antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 
            'lat', 'lng',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['prop_frecuente', 'top_provincia', 'porcentaje_metros', 'diferencia_metros', 'promedio_precio_ciudad', 
                 'promedio_por_mes', 'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 
                 'promedio_precio_hbg_tipo_propiedad', 'count_id_zona', 'count_ciudad', 'puntaje', 
                 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 
                 'promedio_precio_tipo_propiedad_ciudad_gen', 'promedio_precio_hbg_tipo_propiedad_provincia',
                 'varianza_id_zona', 'tam_ambientes', 'metros_cubiertos_normalizados', 'dias_desde_datos',
                 'meses_desde_datos', 'promedio_id_zona_log']

features += features_test

features += cols_tipodepropiedad_ohe + cols_provincia_ohe + cols_zona_ohe


hps = {'alpha': 19.99916025229205,
 'colsample_bytree': 0.65,
 'learning_rate': 0.15,
 'max_depth': int(20.0),
 'n_estimators': int(90.0),
 'test_size': 0.1}

alpha = hps['alpha']
colsample_bytree = hps['colsample_bytree']
max_depth = hps['max_depth']
learning_rate = hps['learning_rate']
n_estimators = hps['n_estimators']
test_size = hps['test_size']

n_estimators = int(hps['n_estimators'])
max_depth = int(hps['max_depth'])


# Filtro columnas
df_XGBoost_fin = utils.filtrar_features(df_train_final_f, features, 'precio')
df_eval_fin = utils.filtrar_features(df_test_final_f, features)

x_train_f, x_test_f, y_train_f, y_test_f = utils.dividir_dataset(df_XGBoost_fin, 'precio', features)


xg_train = xgb.DMatrix(x_train_f, label=y_train_f)
xg_test = xgb.DMatrix(x_test_f, label=y_test_f)

# Modelo - Ver hiperparametros
params = {
    'objective':'reg:squarederror', 
    'colsample_bytree' : colsample_bytree , 
    'learning_rate' : learning_rate,
    'max_depth' : max_depth, 
    'alpha' : alpha,
    'n_estimators' : n_estimators,
    'eval_metric': 'mae'
}

watchlist = [(xg_train, 'train'), (xg_test, 'test')]

# Entrenamiento
xg_reg = xgb.train(params, 
                   xg_train, 
                   num_boost_round=400,
                   evals=watchlist,
                   early_stopping_rounds=10
                  )


# Prediccion
y_pred_test_f = xg_reg.predict(xgb.DMatrix(x_test_f))
y_pred_train_f = xg_reg.predict(xgb.DMatrix(x_train_f))

linear_mae_train_f = utils.MAE(y_train_f, y_pred_train_f)
linear_mae_f = utils.MAE(y_test_f, y_pred_test_f)



print(f"MAE (train): {linear_mae_train_f:.5f}")
print(f"MAE: {linear_mae_f:.5f}")

prediccion_final = xg_reg.predict(xgb.DMatrix(df_eval_fin))
df_test_final_f['target'] = prediccion_final
df_test_final_f[['id', 'target']].to_csv('respuesta22.csv', index = False)

[0]	train-mae:2.16798e+06	test-mae:2.14817e+06
Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 10 rounds.
[1]	train-mae:1.85233e+06	test-mae:1.83954e+06
[2]	train-mae:1.58405e+06	test-mae:1.57918e+06
[3]	train-mae:1.35618e+06	test-mae:1.36214e+06
[4]	train-mae:1.16279e+06	test-mae:1.18192e+06
[5]	train-mae:999053	test-mae:1.03559e+06
[6]	train-mae:860037	test-mae:917059
[7]	train-mae:741232	test-mae:820760
[8]	train-mae:640462	test-mae:744287
[9]	train-mae:554488	test-mae:683466
[10]	train-mae:481530	test-mae:636930
[11]	train-mae:419246	test-mae:600240
[12]	train-mae:365754	test-mae:571849
[13]	train-mae:320298	test-mae:550081
[14]	train-mae:281190	test-mae:533091
[15]	train-mae:248057	test-mae:519944
[16]	train-mae:219714	test-mae:510212
[17]	train-mae:195572	test-mae:502873
[18]	train-mae:174462	test-mae:496886
[19]	train-mae:156750	test-mae:492071
[20]	train-mae:141258	test-mae:488564
[21]	train-mae:1

[205]	train-mae:2563.16	test-mae:474213
[206]	train-mae:2530.77	test-mae:474212
[207]	train-mae:2516.53	test-mae:474213
[208]	train-mae:2506.4	test-mae:474212
[209]	train-mae:2490.79	test-mae:474212
[210]	train-mae:2455.49	test-mae:474211
[211]	train-mae:2447.47	test-mae:474210
[212]	train-mae:2423.23	test-mae:474209
[213]	train-mae:2393.27	test-mae:474210
[214]	train-mae:2389.99	test-mae:474210
[215]	train-mae:2345.39	test-mae:474209
[216]	train-mae:2332.67	test-mae:474209
[217]	train-mae:2302.1	test-mae:474209
[218]	train-mae:2274.37	test-mae:474207
[219]	train-mae:2255.51	test-mae:474206
[220]	train-mae:2244.46	test-mae:474207
[221]	train-mae:2202.9	test-mae:474206
[222]	train-mae:2197.45	test-mae:474206
[223]	train-mae:2188.91	test-mae:474206
[224]	train-mae:2183.04	test-mae:474206
[225]	train-mae:2172.98	test-mae:474205
[226]	train-mae:2160.07	test-mae:474204
[227]	train-mae:2143.07	test-mae:474204
[228]	train-mae:2134.78	test-mae:474204
[229]	train-mae:2122.44	test-mae:474204
[23

### Verificación de Nulos

In [ ]:
df_test_final_f['target'].isnull().any()

In [ ]:
df_test_final_f['target'].isnull().value_counts()

In [ ]:
x_train_f.columns